<center><font size="5"><b>Causal AI: <br> Causal Impact of Player Substitutions on Soccer Team Performance <br> Statsbomb Data Analysis</b></font></center>

In [ ]:
import pathlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from causalai.simulation__ import simulate_matches
from causalai.auxiliaries import *


import dowhy
import networkx as nx
import pygraphviz
# import pgmpy

from pgmpy.models import DiscreteBayesianNetwork
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from pgmpy.inference import CausalInference

import arviz as az
import pymc as pm

In [ ]:
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
seed = sum(map(ord, "eda-simulation"))
rng = np.random.default_rng(seed)

In [ ]:
def R2(y,yhat,ybar):

    SSR = sum((y-yhat)**2)
    SST = sum((y-ybar)**2)
    return 1 - SSR/SST
    

In [ ]:
# =================================== USER INTERACTION -- Global Parameters =================================== #

# -------------------------------- Data -------------------------------- #


# --- Path to the main directory: [None; str]
directory = '/Users/maximilian/Dropbox/Max/52_SoccerCausality' 


# -------------------------------- Model -------------------------------- #

# --- Select the outcomes (Y) that you want to analyze the causal drivers of: ['shots','goals','passes']
OUTCOMES = ["shots","goals","passes"]


# -------------------------------- Figures -------------------------------- #

# --- Want to save the figures? [True; False]
SAVE_FIG = False

# =================================== USER INTERACTION -- Global Parameters =================================== #

<br>

## 00. &emsp; Data Loading / Simulation

In [ ]:
# =================================== 00.1 Load the Data =================================== #

# --- Read in the Data:
df = pd.read_csv(f'{directory}/10_data/data_StatsBomb__byMatch_bySub.csv').astype({'match_id':int})
    

# --- Number of Matches with Substitutions:
matchID__Sub = list(df.loc[df['substitution_replacementNumber__off'].dropna().index,:]['match_id'].unique())

print(f"\n# Match Situations:    {df.shape[0]}")
print(f"# Unique Matches:      {len(df['match_id'].unique())}")
print(f"# Matches with Sub:    {len(matchID__Sub)}")
print(f"Matches with Subs (%): {len(matchID__Sub) / len(df['match_id'].unique()) * 100:.2f}%")
print("\n")
df.head()

In [ ]:
# =================================== 00.2 For Compatibility further down the Road =================================== #

# --- Score Differential in Minute 60:
df['score_diff'] = df['team_off__score60'] - df['team_def__score60']

# --- Indicator whether a substitution was made or not:
df['sub'] = np.where(df['match_id'].isin(matchID__Sub),1,0)


# --- Outcomes per Minute, pre- & post-event:
for team in ['off','def']:

    # --- Passes per Minute:
    df[f'passes_perMin__{team}__pre'] = df[f'passes_total__{team}__pre'] / df['substitution_minute__off']
    df[f'passes_perMin__{team}__post'] = df[f'passes_total__{team}__post'] / df['substitution_minuteDelta__off']
    
    # --- Shots per Minute:
    df[f'shots_perMin__{team}__pre'] = df[f'shots_total__{team}__pre'] / df['substitution_minute__off']
    df[f'shots_perMin__{team}__post'] = df[f'shots_total__{team}__post'] / df['substitution_minuteDelta__off']
    
    # --- Goals per Minute:
    df[f'goals_perMin__{team}__pre'] = df[f'goals_scored__{team}__pre'] / df['substitution_minute__off']
    df[f'goals_perMin__{team}__post'] = df[f'goals_scored__{team}__post'] / df['substitution_minuteDelta__off']


# --- Strength Differential Passes:
df['strength_diff__passes'] = df['passes_total__off__pre'] - df['passes_total__def__pre']

# --- Strength Differential Shots:
df['strength_diff__shots'] = df['shots_total__off__pre'] - df['shots_total__def__pre']

# --- Strength Differential Goals:
df['strength_diff__goals'] = df['goals_scored__off__pre'] - df['goals_scored__def__pre']


# --- Offense Passes per Minute - Differential:
df['passes_perMin__off__DIFF' ] = df['passes_perMin__off__post'] - df['passes_perMin__off__pre']

# --- Offense Shots on Target per Minute - Differential:
df['shots_perMin__off__DIFF' ] = df['shots_perMin__off__post'] - df['shots_perMin__off__pre']

# --- Offense Goals per Minute - Differential:
df['goals_perMin__off__DIFF' ] = df['goals_perMin__off__post'] - df['goals_perMin__off__pre']



In [ ]:
# =================================== 00.3 Features that could influence Strength-Differential =================================== #


[c.split('__off__pre')[0] for c in df.columns if '__off__pre' in c]

In [ ]:
# ==================================== Pre-& Post-Substitution Differentials ==================================== #

"""
    Instead of looking at 'totals', we now compute:
        
        (i)  success rates, and 
        (ii) build a composite index of team-strength, composed of the success rates of
             (ii.a) passes
             (ii.b) shots
             (ii.c) duels
             (ii.d) interceptions
             (ii.e) dribbles

    ... for pre- & post-event segments.
             
"""



# --------------------------------- USER INTERACTION --------------------------------- #

# --- Weighting of Performance Components: [numbers have to sum up to 1!]
wPerformance = {'passes':0.2,'shots':0.2,'duels':0.2,'interceptions':0.2,'dribbles':0.2}

# --------------------------------- USER INTERACTION --------------------------------- #




for sub in ['pre','post']:

    # --- Strength Diff Rate Passes:
    df[f'pass_success_Rate__off__{sub}'] = (df[f'pass_success_length__0_5__off__{sub}'] + df[f'pass_success_length__5_10__off__{sub}'] +
                                        df[f'pass_success_length__10_20__off__{sub}'] + df[f'pass_success_length__20_40__off__{sub}'] +
                                        df[f'pass_success_length__40__off__{sub}']) / df[f'pass_success_length__total__off__{sub}']
    df[f'pass_success_Rate__def__{sub}'] = (df[f'pass_success_length__0_5__def__{sub}'] + df[f'pass_success_length__5_10__def__{sub}'] +
                                        df[f'pass_success_length__10_20__def__{sub}'] + df[f'pass_success_length__20_40__def__{sub}'] +
                                        df[f'pass_success_length__40__def__{sub}']) / df[f'pass_success_length__total__def__{sub}']
    
    # --- Strength Diff Rate Shots:
    df[f'shots_target_Rate__off__{sub}'] = df[f'shots_target__off__{sub}'] / df[f'shots_total__off__{sub}']
    df[f'shots_target_Rate__def__{sub}'] = df[f'shots_target__def__{sub}'] / df[f'shots_total__def__{sub}']
    
    # --- Strength Diff Interceptions:
    df[f'interceptions_Rate__off__{sub}'] = df[f'interceptions_success__off__{sub}'] / df[f'interceptions_total__off__{sub}']
    df[f'interceptions_Rate__def__{sub}'] = df[f'interceptions_success__def__{sub}'] / df[f'interceptions_total__def__{sub}']
    
    # --- Strength Diff Duels:
    df[f'duel_Rate__off__{sub}'] = df[f'duel_success__off__{sub}'] / df[f'duel_total__off__{sub}']
    df[f'duel_Rate__def__{sub}'] = df[f'duel_success__def__{sub}'] / df[f'duel_total__def__{sub}']
    
    # --- Strength Diff Dribble:
    df[f'dribble_Rate__off__{sub}'] = df[f'dribble_success__off__{sub}'] / df[f'dribble_total__off__{sub}']
    df[f'dribble_Rate__def__{sub}'] = df[f'dribble_success__def__{sub}'] / df[f'dribble_total__def__{sub}']
    
    
    
    # --- Strength Diff:
    df[f'strength__off__{sub}'] = (wPerformance[f'passes'] * df[f'pass_success_Rate__off__{sub}'] + 
                                   wPerformance[f'shots'] * df[f'shots_target_Rate__off__{sub}'] + 
                                   wPerformance[f'duels'] * df[f'duel_Rate__off__{sub}'] + 
                                   wPerformance[f'interceptions'] * df[f'interceptions_Rate__off__{sub}'] + 
                                   wPerformance[f'dribbles'] * df[f'dribble_Rate__off__{sub}'])
    df[f'strength__def__{sub}'] = (wPerformance[f'passes'] * df[f'pass_success_Rate__def__{sub}'] + 
                                   wPerformance[f'shots'] * df[f'shots_target_Rate__def__{sub}'] + 
                                   wPerformance[f'duels'] * df[f'duel_Rate__def__{sub}'] + 
                                   wPerformance[f'interceptions'] * df[f'interceptions_Rate__def__{sub}'] + 
                                   wPerformance[f'dribbles'] * df[f'dribble_Rate__def__{sub}'])
    df[f'strength_diff__{sub}'] = df[f'strength__off__{sub}'] - df[f'strength__def__{sub}']






<br>

## 1. &emsp; Descriptive Statistics

In [ ]:
# =================================== 1.1 Some Preliminary Summary Statistics =================================== #

df.describe().T.round(2)

In [ ]:
# =================================== 1.2 Number of Games with Substitutions made =================================== #


# --- Number of Matches:
matchID = df['match_id'].unique().tolist()
# --- Number of Matches with Substitutions:
matchID__Sub = list(df.loc[df['substitution_replacementNumber__off'].dropna().index,:]['match_id'].unique())
# --- Number of Matches with no Substitution:
matchID__noSub = list(df.loc[df['substitution_replacementNumber__off'].isna(),:]['match_id'].unique())



print("\nTreatment Distribution:")
print(f"  No substitution: {len(matchID__noSub)} ({len(matchID__noSub) / len(matchID):.1%})")
print(f"  Substitution:    {len(matchID__Sub)} ({len(matchID__Sub) / len(matchID):.1%})")

In [ ]:
# =================================== 1.3 Inspect Correlations =================================== #


# --- Variables to inspect:
key_vars = list(np.unique(["sub","score_diff"] + [f'{o}_perMin__off__pre' for o in OUTCOMES] + [f'strength_diff__{o}' for o in OUTCOMES]))

# --- Correlation matrix of key variables:
correlation_matrix = df[key_vars].corr()

# --- Plot the Matrix:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0, square=True, fmt=".2f")
plt.title("Correlation Matrix of Key Variables")
plt.tight_layout()
if SAVE_FIG:
    plt.savefig(OUTPUT_DIR / "correlation_matrix.png", dpi=300, bbox_inches="tight")
    print("\nCorrelation matrix saved to output directory.")
plt.show()



In [ ]:
# ================================== 1.4 Zooming in on Score Differential ================================== #

print(f"\nCorrelation between Score Differential & Treatment Assignment: {df["score_diff"].corr(df["sub"]):.3f}")

print("\nCorrelation between Score Differential & Outcomes:\n")
for outcome in [f'{o}_perMin__off__DIFF' for o in OUTCOMES]:
    corr = df["score_diff"].corr(df[outcome])
    print(f"{outcome}: {corr:.3f}")

In [ ]:
# ================================== 1.5 Distribution of Score Differentials ================================== #

# --- Count by Unique Match-ID:
df_plot = df.groupby(["score_diff",'match_id'])["sub"].sum().reset_index().copy()

(df_plot.groupby("score_diff")["sub"].count() / df_plot.shape[0] * 100).plot.bar(
    rot=0, ylabel="Percentage of Total Matches", title="Frequency of observed Score Differentials", xlabel="Score Differential"
);

In [ ]:
# ================================== 1.6 How does Substitution vary with Score Differential? ================================== #

# --- Count by Unique Match-ID:
df_plot = df.groupby(["score_diff",'match_id'])["sub"].sum().reset_index().copy()
df_plot['sub'] = np.where(df_plot['sub'] > 0 ,1,0)

(df_plot.groupby("score_diff")["sub"].sum() / df_plot.groupby("score_diff")["sub"].count()).plot.bar(
    rot=0, ylabel="Share of Matches with Substitution", title="Substitution Rate by Score Differential", xlabel="Score Differential"
);

<br>

## 2. &emsp; A Bayesian Model for Predicting the Impact

The model reads:

\begin{align}
    Y_i \; = \; \alpha \; + \; \beta_{sub} \, T_i + \mathbf{X}_i \, \mathbf{\gamma}^{'} \; + \; \varepsilon_i \quad ,
\end{align}

where $i$ denotes the *event* or *state* of a given match either at the time when a substitution by the lagging team was made, or -- in case the lagging team abstained from making a substitution after minute 60, despite being down by at least one goal -- the state in Minute 59.

$Y$ is the target, i.e. outcome variable. $T = \left\lbrace 0,1\right\rbrace$ is the *treatment variable*, i.e. a dummy that indicates whether the lagging team made a substitution or not.  

$\mathbf{X}$ is a vector of features containing `score_diff`, `strength__off__pre`,`strength__def__pre`. 

In **Section 2.1**, we set $Y$ to be the *difference* of the outcome variable between the its *post-* and *pre-* event value. That is, if $T=1$, it is the *difference* between the value *after* and *before* a substitution was made, respectively the *difference* of the outcome variable *after* and *before* the 60$^\text{th}$ minute, if $T=0$.

Instead, in **Section 2.2**, $Y$ is the observed value of the outcome variable in the *post-event* segment, and not the difference between the *post-* and *pre-event* segment. We thus include $Y_{off, \rightarrow i}$, i.e. the value of the outcome variable of the laggging team recorded up until event $i$, among the features $\mathbf{X}$.
E.g., if $Y$ is `passes_perMin__off__post` (the number of successfull passes per minute in the post-event segment), then $Y_{off, \rightarrow i}$ is the total number of passes made by the lagging team up until the minute of event $i$.

In [ ]:
df.columns.tolist()

<br>

### 2.1 &emsp; $Y$: &ensp; Difference between post- & pre-Event Performance


Here, $Y$ is the *difference* of the outcome variable between the its *post-* and *pre-* event value. That is, if $T=1$, it is the *difference* between the value *after* and *before* a substitution was made, respectively the *difference* of the outcome variable *after* and *before* the 60$^\text{th}$ minute, if $T=0$.

In [ ]:
# --------------------------------- USER INTERACTION --------------------------------- #

# --- Which is the outcome (Y) you are interested in to analyze? ['passes','shots','goals']
target = "shots"

# --- Transform the target? [None; 'log']
target_transform = None

# --- Set the Features you want to include:
features = ['sub','score_diff','strength__off__pre','strength__def__pre']

# --------------------------------- USER INTERACTION --------------------------------- #


# --- Outcome Label:
outcome_label = f'{target}_perMin__off__DIFF' 

# --- Regression Matrix:
df_reg = df[features+[outcome_label]].dropna().copy()

# --- If log-Transformation was requested:
if target_transform == 'log':
    df_reg[outcome_label] = np.where(df_reg[outcome_label] == 0,1,df_reg[outcome_label])
    df_reg[outcome_label] = np.log(df_reg[outcome_label])
    df_reg = df_reg.dropna()



COORDS = {
    'obs_id': df_reg.index,
    'features': features,
}


print(df_reg.shape)

In [ ]:

# --- Get the Target:
s_plot = df_reg[outcome_label].copy()

fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot.values,ax=ax)
plt.ylabel('')
plt.xlabel('')
plt.legend('')
plt.show()

# --- Some Summary Statistics:

print(f'\nMean:      {s_plot.mean(): .2f}')
print(f'SD:        {s_plot.std(): .2f}')
print(f'Skew:      {s_plot.skew(): .2f}')
print(f'Kurtosis:  {s_plot.kurt()+3: .2f}')


In [ ]:

with pm.Model(coords=COORDS) as CausalSoccer_model:
    
    # ================================== Set the Data ================================== #
    X = pm.Data('X',df_reg[features].to_numpy(),dims=('obs_id','features'))
    Y = pm.Data('Y',df_reg[outcome_label].to_numpy(),dims='obs_id')
    

    # ================================== Set the priors ================================== #

    # -------------------------------- Alpha -------------------------------- # 
   
    # --- Hyperparameter: Standard-Deviation:
    sigma__alpha = pm.HalfNormal('sigma__alpha',sigma=3)
    #sigma__alpha = pm.Exponential('sigma__alpha',lam=1)
    

    # --- League-Specific
    alpha = pm.Normal('alpha', mu=df_reg[outcome_label].mean(), sigma=sigma__alpha)


    # -------------------------------- Betas -------------------------------- # 

    # --- Variation in the loadings:
    sigma_betas = pm.HalfNormal('sigma_betas',sigma=3,dims='features')
    #sigma_betas = pm.Exponential('sigma_betas',lam=0.5,dims='features')

    # --- Loadings: Pooled across all players & leagues
    betas = pm.Normal('betas',mu=0,sigma=sigma_betas,dims='features')


    # ---------------------- Prior for the Variance of the Student-T used to model the Target ------------------- # 
    # ---> set it a bit looser
    sigma_lik = pm.HalfNormal('sigma_lik',sigma=3)
    #sigma_lik = pm.Exponential('sigma_lik',lam=0.5)

    
    # ---------------------- Degrees of Freedom of Student-T used to model the Target ------------------- # 
    nu_lik = pm.HalfNormal('nu_lik',sigma=0.1)
    #nu_lik = pm.Exponential('nu_lik',lam=0.1)


    
    # ================================== Compute the Conditional Mean ================================== #
    dgp = pm.Deterministic('dgp', alpha + X @ betas, dims='obs_id')


    # ================================== Likelihood ================================== #

    likelihood = pm.StudentT('likelihood',
                             mu=dgp, 
                             sigma=sigma_lik, 
                             nu=nu_lik,
                             observed=Y, 
                             dims='obs_id')

    

In [ ]:
pm.model_to_graphviz(CausalSoccer_model)

In [ ]:
with CausalSoccer_model:
    idata = pm.sample_prior_predictive(draws=5000)

In [ ]:



fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot,color='firebrick',label='obs',ax=ax, stat='density')
sns.histplot(x=idata['prior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
             color='darkgreen',label='prior preds',ax=ax, stat='density')
plt.ylabel('')
plt.xlabel('')
plt.legend()
plt.title('Prior Predictive')
plt.show()


# --- Some Summary Statistics:
s_priorPred = pd.Series(idata['prior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1))
print(f'\nMean:      {s_priorPred.mean(): .2f}')
print(f'SD:        {s_priorPred.std(): .2f}')
print(f'Skew:      {s_priorPred.skew(): .2f}')
print(f'Kurtosis:  {s_priorPred.kurt()+3: .2f}')


In [ ]:
N_chains = 4

with CausalSoccer_model:
    idata = pm.sample(nuts_sampler="numpyro", target_accept=0.99, chains=N_chains,draws=1000)


In [ ]:
# ===================================== Make Predictions ===================================== #

with CausalSoccer_model:
    idata.extend(pm.sample_posterior_predictive(idata))

In [ ]:
# ===================================== Evaluate the Fitting: Just a 'by-foot' workaround ===================================== #
if 1==2:
    # --- Extract the Estimates:
    alpha__samples = idata['posterior']['alpha'].stack(samples=('chain','draw')).to_numpy()
    beta__samples = idata['posterior']['betas'].stack(samples=('chain','draw')).to_numpy()
    
    
    # --- Predict:
    df_Yhat = pd.DataFrame(np.nan,index=df_reg.index,columns=[f'draw__{n}' for n in range(len(alpha__samples))])
    for n in tqdm(range(len(alpha__samples))):
        df_Yhat.loc[:,f'draw__{n}'] = alpha__samples[n] + df_reg[features] @ beta__samples[:,n]
    
    
    # --- Evaluate:
    print(f'\nGoodness of Fit (R^2): {R2(idata['constant_data']['Y'].to_numpy(),
                                         df_Yhat.median(axis=1).values,
                                         idata['constant_data']['Y'].mean().to_numpy()):.2f}')


In [ ]:
# ===================================== Plot the Posterior Predictive vs. the Target ===================================== #


# --- Get the Target:
s_plot = df_reg[outcome_label].copy()



fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot,color='firebrick',label='obs',ax=ax, stat='density')
sns.histplot(x=idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
             color='darkgreen',label='preds',ax=ax, stat='density')
plt.ylabel('')
plt.xlabel('')
plt.legend('')
plt.show()


s_postPred = pd.Series(idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1))
print(f'\nMean:      {s_postPred.mean(): .2f}')
print(f'SD:        {s_postPred.std(): .2f}')
print(f'Skew:      {s_postPred.skew(): .2f}')
print(f'Kurtosis:  {s_postPred.kurt()+3: .2f}')



In [ ]:
# ===================================== Evaluate the Fitting: the Quick Alternative ===================================== #


print(f'\nGoodness of Fit (R^2): {R2(s_plot,
                                     idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
                                     s_plot.mean()):.2f}')

In [ ]:
features

In [ ]:
# ===================================== Evaluate the Treatment Effect: Plot '\beta_{sub}' ===================================== #


# --------------------------- USER INTERACTION --------------------------- #

# --- Confidence Interval: ]0.5,1]
pick_q = 0.95

# --- Which feature?
pick_feature = 'sub'

# --------------------------- USER INTERACTION --------------------------- #


beta__samples = idata['posterior']['betas'][:,:,features.index(pick_feature)].to_numpy().flatten()



fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=beta__samples,ax=ax)
plt.ylabel('')
plt.xlabel('')
plt.legend('')
plt.title(f'Substitution (T=1): Increase in # of {outcome_label.capitalize()}')
plt.show()

# --- Some Summary Statistics:


print('\n')
print(f'Lower Bound  {(1-pick_q)/2*100:.1f}%:  {np.quantile(beta__samples,q=(1-pick_q)/2): .2f}')
print(f'Mean:               {beta__samples.mean(): .2f}')
print(f'Upper Bound {(1-(1-pick_q)/2)*100:.1f}%:  {np.quantile(beta__samples,q=1-(1-pick_q)/2): .2f}')


<br>

### 2.2 &emsp; $Y$: &ensp; Post-Event Performance


$Y$ is the observed value of the outcome variable in the *post-event* segment, and not the difference between the *post-* and *pre-event* segment. We thus include $Y_{off, \rightarrow i}$, i.e. the value of the outcome variable of the laggging team recorded up until event $i$, among the features $\mathbf{X}$.


In [ ]:
# --------------------------------- USER INTERACTION --------------------------------- #

# --- Which is the outcome (Y) you are interested in to analyze? ['passes','shots','goals']
target = "passes"

# --- Transform the target? [None; 'log']
target_transform = 'log'

# --- Set the Features you want to include:
features = ['sub','score_diff','strength__off__pre','strength__def__pre']

# --------------------------------- USER INTERACTION --------------------------------- #


# --- Outcome Label:
outcome_label = f'{target}_perMin__off__post'

# --- Add the pre-Event Equivalent to the Features:
features += [f'{target}_perMin__off__pre']

# --- Regression Matrix
df_reg = df[features+[outcome_label]].dropna().copy()
df_reg = df_reg[df_reg[outcome_label] != 0]

# --- If log-Transformation was requested
if target_transform == 'log':
    df_reg[outcome_label] = np.log(df_reg[outcome_label])



COORDS = {
    'obs_id': df_reg.index,
    'features': features,
}


df_reg.shape

In [ ]:

# --- Get the Target:
s_plot = df_reg[outcome_label].copy()

fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot.values,ax=ax)
plt.ylabel('')
plt.xlabel('')
plt.legend('')
plt.show()

# --- Some Summary Statistics:

print(f'\nMean:      {s_plot.mean(): .2f}')
print(f'SD:        {s_plot.std(): .2f}')
print(f'Skew:      {s_plot.skew(): .2f}')
print(f'Kurtosis:  {s_plot.kurt()+3: .2f}')


In [ ]:


with pm.Model(coords=COORDS) as CausalSoccer_model:


    # ================================== Set the Data ================================== #
    X = pm.Data('X',df_reg[features].to_numpy(),dims=('obs_id','features'))
    Y = pm.Data('Y',df_reg[outcome_label].to_numpy(),dims='obs_id')
    

    # ================================== Set the priors ================================== #

    # -------------------------------- Alpha -------------------------------- # 
   
    # --- Hyperparameter: Standard-Deviation:
    #sigma__alpha = pm.Exponential('sigma__alpha',lam=0.5)
    sigma__alpha = pm.HalfNormal('sigma__alpha',sigma=3)
    #mu__alpha = pm.Normal('mu__alpha',mu=0,sigma=2)
    mu__alpha = s_plot.mean()
    

    # --- League-Specific
    alpha = pm.Normal('alpha', mu=mu__alpha, sigma=sigma__alpha)


    # -------------------------------- Betas -------------------------------- # 

    # --- Variation in the loadings:
    sigma_betas = pm.HalfNormal('sigma_betas',sigma=3,dims='features')
    #sigma_betas = pm.Exponential('sigma_betas',lam=0.5,dims='features')

    # --- Loadings: Pooled across all players & leagues
    betas = pm.Normal('betas',mu=0,sigma=sigma_betas,dims='features')


    # ---------------------- Prior for the Variance of the Target ------------------- # 
    # ---> set it a bit looser
    sigma_lik = pm.HalfNormal('sigma_lik',sigma=5)
    #sigma_lik = pm.Exponential('sigma_lik',lam=0.1)


    
    # ================================== Compute the Conditional Mean ================================== #
    
    dgp = pm.Deterministic('dgp', alpha + X @ betas, dims='obs_id')

    

    # ================================== Likelihood ================================== #

    if not target_transform:

        
        likelihood = pm.LogNormal('likelihood',
                                   mu=dgp, 
                                   sigma=sigma_lik, 
                                   observed=Y, 
                                   dims='obs_id')
            
    elif target_transform == 'log':

        
        if target == 'passes':

            # ---------------------- Prior for the Parameters describing the Kurtosis of the Skewed-Student-T ------------------- # 
            a_SkewStudent = pm.HalfNormal('a_SkewStudent',sigma=3)
            b_SkewStudent = pm.HalfNormal('b_SkewStudent',sigma=3)
            #a_SkewStudent = pm.Exponential('a_SkewStudent',lam=0.1)
            #b_SkewStudent = pm.Exponential('b_SkewStudent',lam=0.1)

            
            
            likelihood = pm.SkewStudentT('likelihood',
                                        mu=dgp, 
                                        a=a_SkewStudent,
                                        b=b_SkewStudent,
                                        sigma=sigma_lik,
                                        observed=Y, 
                                        dims='obs_id')

        elif target == 'shots':
        
            likelihood = pm.Normal('likelihood',
                                    mu=dgp, 
                                    sigma=sigma_lik, 
                                    #sigma=2, 
                                    observed=Y, 
                                    dims='obs_id')


    


    

In [ ]:
pm.model_to_graphviz(CausalSoccer_model)

In [ ]:
with CausalSoccer_model:
    idata = pm.sample_prior_predictive(draws=5000)

In [ ]:



fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot,color='firebrick',label='obs',ax=ax, stat='density')
sns.histplot(x=idata['prior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
             color='darkgreen',label='prior preds',ax=ax, stat='density')
plt.ylabel('')
plt.xlabel('')
plt.legend()
plt.title('Prior Predictive')
plt.show()


# --- Some Summary Statistics:
s_priorPred = pd.Series(idata['prior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1))
print(f'\nMean:      {s_priorPred.mean(): .2f}')
print(f'SD:        {s_priorPred.std(): .2f}')
print(f'Skew:      {s_priorPred.skew(): .2f}')
print(f'Kurtosis:  {s_priorPred.kurt()+3: .2f}')


In [ ]:
N_chains = 4

with CausalSoccer_model:
    idata = pm.sample(nuts_sampler="numpyro", target_accept=0.99, chains=N_chains,draws=1000)
    #idata = pm.sample(nuts_sampler="pymc", target_accept=0.99, chains=N_chains,draws=1000)


In [ ]:
# ===================================== Make Predictions ===================================== #

with CausalSoccer_model:
    idata.extend(pm.sample_posterior_predictive(idata))

In [ ]:
# ===================================== Evaluate the Fitting: Just a 'by-foot' workaround ===================================== #
if 1==2:
    # --- Extract the Estimates:
    alpha__samples = idata['posterior']['alpha'].stack(samples=('chain','draw')).to_numpy()
    beta__samples = idata['posterior']['betas'].stack(samples=('chain','draw')).to_numpy()
    
    
    # --- Predict:
    df_Yhat = pd.DataFrame(np.nan,index=df_reg.index,columns=[f'draw__{n}' for n in range(len(alpha__samples))])
    for n in tqdm(range(len(alpha__samples))):
        df_Yhat.loc[:,f'draw__{n}'] = alpha__samples[n] + df_reg[features] @ beta__samples[:,n]
    
    
    # --- Evaluate:
    print(f'\nGoodness of Fit (R^2): {R2(idata['constant_data']['Y'].to_numpy(),
                                         df_Yhat.median(axis=1).values,
                                         idata['constant_data']['Y'].mean().to_numpy()):.2f}')


In [ ]:
# ===================================== Plot the Posterior Predictive vs. the Target ===================================== #


# --- Get the Target:
s_plot = df_reg[outcome_label].copy()



fig,ax=plt.subplots(figsize=(8,5))
sns.histplot(x=s_plot,color='firebrick',label='obs',ax=ax, stat='density')
sns.histplot(x=idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
             color='darkgreen',label='preds',ax=ax, stat='density')
plt.ylabel('')
plt.xlabel('')
plt.legend('')
plt.show()


s_postPred = pd.Series(idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1))
print(f'\nMean:      {s_postPred.mean(): .2f}')
print(f'SD:        {s_postPred.std(): .2f}')
print(f'Skew:      {s_postPred.skew(): .2f}')
print(f'Kurtosis:  {s_postPred.kurt()+3: .2f}')



In [ ]:
# ===================================== Evaluate the Fitting: the Quick Alternative ===================================== #


print(f'\nGoodness of Fit (R^2): {R2(s_plot,
                                     idata['posterior_predictive']['likelihood'].stack(samples=('chain','draw')).median(axis=1),
                                     s_plot.mean()):.2f}')

In [ ]:
features

In [ ]:
# ===================================== Evaluate the Treatment Effect: Plot '\beta_{sub}' ===================================== #


# --------------------------- USER INTERACTION --------------------------- #

# --- Confidence Interval: ]0.5,1]
pick_q = 0.95

# --- Which feature?
pick_feature = 'sub'

# --------------------------- USER INTERACTION --------------------------- #


beta__samples = idata['posterior']['betas'][:,:,features.index(pick_feature)].to_numpy().flatten()



fig,ax=plt.subplots(figsize=(7,5))
sns.histplot(x=beta__samples,ax=ax)
plt.ylabel('')
plt.xlabel('')
plt.legend('')
#plt.title(f'Substitution (T=1): Increase in # of {outcome_label.capitalize()}')
plt.show()

# --- Some Summary Statistics:


print('\n')
print(f'Lower Bound  {(1-pick_q)/2*100:.1f}%:  {np.quantile(beta__samples,q=(1-pick_q)/2): .2f}')
print(f'Mean:               {beta__samples.mean(): .2f}')
print(f'Upper Bound {(1-(1-pick_q)/2)*100:.1f}%:  {np.quantile(beta__samples,q=1-(1-pick_q)/2): .2f}')


In [ ]:
#fig.savefig(f'{directory}/20_figures/PostDist__sub__logPPM.png')